In [2]:
#import Pkg; Pkg.add("Distributions")

    Updating registry at `C:\Users\peren\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
   Installed DensityInterface ─ v0.4.0
   Installed FillArrays ─────── v0.13.2
   Installed PDMats ─────────── v0.11.8
   Installed StatsFuns ──────── v0.9.18
   Installed QuadGK ─────────── v2.4.2
   Installed Distributions ──── v0.25.55
    Updating `C:\Users\peren\.julia\environments\v1.7\Project.toml`
  [31c24e10] + Distributions v0.25.55
    Updating `C:\Users\peren\.julia\environments\v1.7\Manifest.toml`
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.55
  [1a297f60] + FillArrays v0.13.2
  [90014a1f] + PDMats v0.11.8
  [1fd47b50] + QuadGK v2.4.2
  [79098fc4] + Rmath v0.7.0
  [4c63d2b9] + StatsFuns v0.9.18
  [f50d1b31] + Rmath_jll v0.3.0+0
  [4607b0f0] + SuiteSparse
Precompiling project...
  ✓ DensityInterface
  ✓ PDMats
  ✓ QuadGK
  ✓ FillArrays
  ✓ Rmath_jll
  ✓ Rmath
  ✓ StatsFuns
  

In [3]:
using JuMP, Cbc, Random, Distributions

# we'll build the model using number indices
# but we'll display the solution with names
cities = 1:4
states = 1:4
citynames = [:Madison,:Chicago,:DesMoines,:Minneapolis]
statenames = [:Wisconsin,:Illinois,:Iowa,:Minnesota]
cost = [
    15  45  55  70;
    40  10  50  115;
    40  20  65  75;
    65  112 85 10]

capacity = [1200, 1200, 1200, 1200] # U
unmet_pen = [180,180,180,180] # q

demandmean = [835, 540, 545, 455]
demandstdev = [200, 50, 75, 125]
numscen = 100
scenarios = 1:100

## Following command makes sure the "random" demands we create
## for scenarios are the same every time we run the code
Random.seed!(425) 
demandscens=Array{Float64}(undef,4,numscen) # Total 4 * 100 scenarios
for s in scenarios
    for i in states
        demandscens[i,s]=round(rand(Normal(demandmean[i],demandstdev[i])),digits=0)
    end
end

fixedcost = 30000

### First build and solve the deterministic model
### where we use the mean demands as estimates
detm = Model()

@variable(detm, 
    amountsent[cities,states] >= 0) # x
@variable(detm, 
    open[cities], Bin) # y
@variable(detm,
    shortfall[states] >= 0) # z

@objective(detm, Min,
    fixedcost*sum(open[i] for i in cities)
    + sum(cost[i,j]amountsent[i,j] 
        for i in cities for j in states)
    + sum(unmet_pen[j]*shortfall[j] 
        for j in states)) # f * y + c * x + q * z

@constraint(detm, meetdemand[j in states],
    sum(amountsent[i,j] for i in cities)
       + shortfall[j] == demandmean[j]) # x + z = d

@constraint(detm, brewerylimit[i in cities],
    sum(amountsent[i,j] for j in states) 
            <= capacity[i]*open[i]) # x <= U * y
    
set_optimizer(detm, Cbc.Optimizer)
set_silent(detm)
optimize!(detm)

## display and store solution
determsol = zeros(4)
println("\nDeterministic Model Total cost: \$", 
    objective_value(detm))
println("Deterministic solution: Open brewery in:")
for i in cities
    if value(open[i]) > 0.1
        println(citynames[i])
        determsol[i]=1
    end
end


Deterministic Model Total cost: $139275.0
Deterministic solution: Open brewery in:
Madison
Chicago

Stochastic solution: Total expected cost= $139662.75
Open brewery in:
Madison
Chicago
Minneapolis

Deterministic solution: Total expected cost= $147811.15000000002


In [ ]:
### Now build and solve the stochastic model

stochm = Model()

## using s_... for all variable and constraint names in
## stochastic model to differentiate from those in deterministic model
@variable(stochm, 
    s_amountsent[cities,states,scenarios] >= 0) # x
@variable(stochm, 
    s_open[cities], Bin) # y
@variable(stochm,
    s_shortfall[states,scenarios] >= 0) # z

@objective(stochm, Min,
    fixedcost*sum(s_open[i] for i in cities)
    + 1/numscen*sum(cost[i,j]s_amountsent[i,j,s] 
        for i in cities for j in states for s in scenarios)
    + 1/numscen*sum(unmet_pen[j]*s_shortfall[j,s] 
        for j in states for s in scenarios)) # f * y + 1/|S| * c * x + 1/|S| * q * z

@constraint(stochm, s_meetdemand[j in states, s in scenarios],
    sum(s_amountsent[i,j,s] for i in cities)
       + s_shortfall[j,s] == demandscens[j,s]) # x + z = d

@constraint(stochm, s_brewerylimit[i in cities, s in scenarios],
    sum(s_amountsent[i,j,s] for j in states) 
            <= capacity[i]*s_open[i]) # x <= U * y
    
set_optimizer(stochm, Cbc.Optimizer)
set_silent(stochm)
optimize!(stochm)

println("\nStochastic solution: Total expected cost= \$", 
    objective_value(stochm))
println("Open brewery in:")
for i in cities
    if value(s_open[i]) > 0.1
        println(citynames[i])
    end
end

## Now let's evaluate expected cost of solution we
## obtained from deterministic model

@constraint(stochm, fixsol[i in cities],
    s_open[i]==determsol[i])

optimize!(stochm)

println("\nDeterministic solution: Total expected cost= \$", 
    objective_value(stochm))

In [4]:
demandscens

4×100 Matrix{Float64}:
 972.0  1079.0  874.0  725.0  714.0  …  728.0  947.0  892.0  172.0  486.0
 520.0   549.0  674.0  480.0  552.0     579.0  541.0  587.0  578.0  564.0
 582.0   354.0  582.0  701.0  635.0     469.0  573.0  604.0  496.0  615.0
 399.0   257.0  530.0  475.0  881.0     546.0  482.0  398.0  298.0  549.0